In [1]:
# !pip install -q datasets
from datasets import load_dataset
from transformers import AutoTokenizer

import torch
from torch.utils.data import DataLoader


ModuleNotFoundError: No module named 'datasets'

In [2]:
torch.cuda.is_available()

True

In [3]:
import sys
sys.path.append("/home/s0001666/Working_dir/my_repos/GPT_my/gpt_impl/src/")


In [4]:
from gptmini.data.dataloader import MyGPTDataset, gpt_collate_fn

In [5]:
ds = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")
train = ds["train"]
val   = ds["validation"]

print(len(train), len(val))



1801350 3760


/home/s0001666/.local/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [6]:
print(train[3]["text"][:300])

 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in Janu


In [7]:
train[0]

{'text': ''}

In [8]:
train[3]

{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'}

In [9]:

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

# GPT-2 tokenizer has no pad token by default; if you ever need padding, do this:
tokenizer.pad_token = tokenizer.eos_token

# quick check
s = "Test message that I want to encode."
ids = tokenizer.encode(s)
print(ids, tokenizer.decode(ids))


2026-01-26 14:54:19.698908: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-26 14:54:19.723622: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-26 14:54:19.723668: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-26 14:54:19.723689: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-26 14:54:19.729295: I tensorflow/core/platform/cpu_feature_g

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2026-01-26 14:54:20.294427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

[14402, 3275, 326, 314, 765, 284, 37773, 13] Test message that I want to encode.


### Make streams 

In [10]:
eos = tokenizer.eos_token_id

In [11]:
train_texts = [t for t in ds["train"]["text"] if t.strip()]
val_texts   = [t for t in ds["validation"]["text"] if t.strip()]

In [12]:
len(train_texts), len(val_texts)

(1165029, 2461)

In [13]:
def build_stream(texts, batch_size=1024):
    stream = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, add_special_tokens=False).input_ids  # list[list[int]]
        for ids in enc:
            stream.extend(ids)
            stream.append(eos)
    return torch.tensor(stream, dtype=torch.long)

In [14]:
val_stream   = build_stream(val_texts)

In [15]:
train_stream = build_stream(train_texts)


Token indices sequence length is longer than the specified maximum sequence length for this model (1063 > 1024). Running this sequence through the model will result in indexing errors


In [16]:

print(train_stream.shape, val_stream.shape)

torch.Size([119085169]) torch.Size([249750])


In [17]:
train_dataset = MyGPTDataset(train_stream, 256)
eval_dataset = MyGPTDataset(val_stream, 256)

In [18]:
train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True, drop_last=True,
    num_workers=2, pin_memory=True, collate_fn=gpt_collate_fn
)

eval_loader = DataLoader(
    eval_dataset, batch_size=32, shuffle=False, drop_last=False,
    num_workers=2, pin_memory=True, collate_fn=gpt_collate_fn
)

In [19]:
x = next(iter(train_loader))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
x.shape

torch.Size([16, 257])

## Train

In [21]:
from gptmini.model.gpt import GPTmini

from gptmini.trainer import train_step, eval_step, TransLoss

In [22]:
# hyperparameters 
vocab_len = len(tokenizer)
max_len = 256
input_dim = 256
n_layers = 4
n_heads = 4
feature_dim = 256
ffn_dim = input_dim * 4
ffn_dropout = 0.1
attn_dropout = 0.1

In [23]:
model = GPTmini(
    max_len,
        vocab_len,
        input_dim,
        n_layers,
        n_heads,
        feature_dim,
        ffn_dim,
        ffn_dropout=0.0,
        attn_dropout=0.0,
        emb_type='rotary',           
).cuda()

In [ ]:

loss_fn = TransLoss(label_smoothing=0.1)
optim = torch.optim.AdamW(
    model.parameters(),
    weight_decay=0.1,
    lr=3e-4,                
    betas=(0.9, 0.95),
    eps=1e-8,
)



In [25]:
d_model = input_dim
warmup_steps = 1000

def noam_lr(step: int):
    step = max(step, 1)
    return (d_model ** -0.5) * min(step ** -0.5, step * (warmup_steps ** -1.5))

In [26]:
# scheduler = torch.optim.lr_scheduler.LambdaLR(
#     optim, lr_lambda=lambda step: noam_lr(step)
# )

In [27]:
from collections import OrderedDict

def count_params(p_iter):
    return sum(p.numel() for p in p_iter)

def group_param_counts(model):
    """
    Groups params by Transformer blocks (encoder/decoder FFN/attn/norms + embeddings/head/PE).
    Adjust rules if your attribute names differ.
    """
    rules = OrderedDict({
        # top-level
        "embeddings/source":      {"all": ["source_embedding"]},
        "embeddings/target":      {"all": ["target_embedding"]},
        "positional_encoding":    {"all": ["PE."]},          # buffer-only usually => 0 trainable
        "lm_head":                {"all": ["lm_head"]},

        # encoder
        "encoder/attn":           {"all": ["encoder."], "any": [".mha."]},
        "encoder/ffn":            {"all": ["encoder."], "any": [".ffn."]},
        "encoder/norms":          {"all": ["encoder."], "any": ["norm"]},

        # decoder
        "decoder/self_attn":      {"all": ["decoder."], "any": [".sa."]},
        "decoder/cross_attn":     {"all": ["decoder."], "any": [".ca."]},
        "decoder/ffn":            {"all": ["decoder."], "any": [".ffn."]},
        "decoder/norms":          {"all": ["decoder."], "any": ["norm"]},
    })

    # init buckets
    buckets = OrderedDict((k, {"total": 0, "trainable": 0}) for k in rules.keys())
    buckets["other"] = {"total": 0, "trainable": 0}

    def match(name, rule):
        if "all" in rule and not all(s in name for s in rule["all"]):
            return False
        if "any" in rule and not any(s in name for s in rule["any"]):
            return False
        return True

    for name, p in model.named_parameters():
        placed = False
        for gname, rule in rules.items():
            if match(name, rule):
                buckets[gname]["total"] += p.numel()
                if p.requires_grad:
                    buckets[gname]["trainable"] += p.numel()
                placed = True
                break
        if not placed:
            buckets["other"]["total"] += p.numel()
            if p.requires_grad:
                buckets["other"]["trainable"] += p.numel()

    return buckets

# ---- usage ----
buckets = group_param_counts(model)
for k, v in buckets.items():
    print(f"{k:22s} total={v['total']:,}  trainable={v['trainable']:,}")

print("\nGrand total:", sum(v["total"] for v in buckets.values()))


embeddings/source      total=0  trainable=0
embeddings/target      total=0  trainable=0
positional_encoding    total=0  trainable=0
lm_head                total=0  trainable=0
encoder/attn           total=0  trainable=0
encoder/ffn            total=0  trainable=0
encoder/norms          total=0  trainable=0
decoder/self_attn      total=0  trainable=0
decoder/cross_attn     total=0  trainable=0
decoder/ffn            total=0  trainable=0
decoder/norms          total=0  trainable=0
other                  total=29,204,305  trainable=29,204,305

Grand total: 29204305


In [28]:
padding_idx = -1
num_epochs = 10
log_every = int(len(train_loader) / 10)

In [ ]:
def make_cosine_warmup_scheduler(optim, warmup_steps, total_steps, min_lr_ratio=0.1):
    def lr_lambda(step):
        step = max(step, 1)
        if step <= warmup_steps:
            return step / warmup_steps
        progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
        cosine = 0.5 * (1.0 + math.cos(math.pi * progress))
        return min_lr_ratio + (1.0 - min_lr_ratio) * cosine

    return torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda)

scheduler = make_cosine_warmup_scheduler(optim, warmup_steps=warmup_steps, total_steps=len(train_loader) * num_epochs)

In [29]:
global_step = 0
t_losses = []
v_losses = []
for epoch in range(1, num_epochs + 1):
    # ---- train ----
    running = 0.0
    n = 0
    for step, batch in enumerate(train_loader, start=1):
        global_step += 1
        loss = train_step(model, batch, optim, loss_fn, padding_idx, scheduler)
        running += loss
        n += 1

        if step % log_every == 0:
            print(f"epoch {epoch} step {step} | train_loss {running/n:.4f} | lr {optim.param_groups[0]['lr']:.7f}")

    train_loss = running / max(n, 1)
    t_losses.append(train_loss)
    
    # ---- valid ----
    val_sum = 0.0
    m = 0
    for batch in eval_loader:
        val_sum += eval_step(model, batch, loss_fn, padding_idx)
        m += 1
    val_loss = val_sum / max(m, 1)
    v_losses.append(val_loss)
    print(f"\nEPOCH {epoch} DONE | train_loss {train_loss:.4f} | val_loss {val_loss:.4f}\n")

   

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


epoch 1 step 2907 | train_loss 6.5666 | lr 0.0005796
epoch 1 step 5814 | train_loss 6.1687 | lr 0.0004098
epoch 1 step 8721 | train_loss 5.9820 | lr 0.0003346
epoch 1 step 11628 | train_loss 5.8660 | lr 0.0002898
epoch 1 step 14535 | train_loss 5.7853 | lr 0.0002592
epoch 1 step 17442 | train_loss 5.7236 | lr 0.0002366
epoch 1 step 20349 | train_loss 5.6753 | lr 0.0002191
epoch 1 step 23256 | train_loss 5.6351 | lr 0.0002049
epoch 1 step 26163 | train_loss 5.6007 | lr 0.0001932
epoch 1 step 29070 | train_loss 5.5712 | lr 0.0001833


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: shape '[32, 32, 4, 64]' is invalid for input of size 8192

In [ ]:
l = torch.nn.Embedding(vocab_len, input_dim)
l(x).shape

In [ ]:
# ---- sample generation ----
if (tokenizer_de_en is not None) and (epoch % gen_every == 0):
    model.eval()
    src_ids, tgt_ids = next(iter(gpt_dataloader_eval))
    src_ids = src_ids[:gen_k].to('cuda')
    tgt_ids = tgt_ids[:gen_k]

    pred_ids = model.generate(src_ids, bos_token=bos_token_id, eos_token=eos_token_id, strategy="beam")

    for i in range(min(gen_k, src_ids.size(0))):
        src_txt = tokenizer_de_en.decode(src_ids[i].detach().cpu(), skip_special_tokens=False)
        tgt_txt = tokenizer_de_en.decode(tgt_ids[i].detach().cpu(), skip_special_tokens=False)
        pred_txt = tokenizer_de_en.decode(pred_ids[i].detach().cpu(), skip_special_tokens=False)
        print(f"[sample {i}] SRC : {src_txt}")
        print(f"[sample {i}] TGT : {tgt_txt}")
        print(f"[sample {i}] PRED: {pred_txt}")
        print()
